In [2]:
import tomotopy as tp
import pandas as pd
import numpy as np
import sys
import nltk
from nltk import word_tokenize, RegexpTokenizer,PunktSentenceTokenizer, sent_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

### 1. Load the model and the abstracts file 

In [3]:
model = tp.LDAModel.load("lda_model150.bin")

In [10]:
data = pd.read_csv("abstracts_eng.csv")
collection = list(data['abstract'])

In [20]:
data.head(3)

,abstract,id,title
0,The goal of this note is to introduce new clas...,188444.0,Asymptotic versions for operators and operator...
1,We will review a Lemma published by Ran Raz in...,188623.0,Some remarks on a lemma of Ran Raz
2,China’s growing influence in Europe has the po...,346708.0,China as a Stress Test for Europe’s Coherence


In [5]:
# Example collection
toy_collection = ['hello hi I am a Data Science STUDENT here with Giulia and Andreas!U.S.A, United States of AMerica dixterochlomaterine hreoihso my dog ran away yesterday?! I will fly, to space tomorrow...',
                  'space tomorrow space tomorrow if the dog ran away fly to space',
                  ' fly to space  with my dog ran away']

### 2. Pre-process the text

The pre-processing comprises 2 steps: 
- tokenisation, stopwords-removal, stemming (optional), lemmatizing(optional)
- ngrams modeling (bigrams & trigrams supported)

Note: the first step is only dependent on a single input document, while ngrams modeling depends on the whole collection

In [6]:
# Step 1
word_stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
stop_words = stopwords.words('english')
def normalisation(document, stemming = True, lemmatising = True, min_word_len = 3):
    tokens = gensim.utils.simple_preprocess(str(document), deacc=True, max_len = sys.maxsize)
    cleaned = [word for word in tokens if word not in stop_words]
    if stemming:
        cleaned = [word_stemmer.stem(word) for word in cleaned]
    if lemmatising:
        cleaned = [lemmatiser.lemmatize(word) for word in cleaned]
    cleaned = [word for word in cleaned if (min_word_len<=len(word))]
    return cleaned

In [7]:
# Step 2
def ngram(cleaned_docs, do_trigram = True, min_count_bigram = 5, threshold_bigram = 50, min_count_trigram = 5, threshold_trigram=50):
    #Bigrams
    bigram = gensim.models.Phrases(cleaned_docs, min_count= min_count_bigram, threshold=threshold_bigram) 
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    output = [bigram_mod[doc] for doc in cleaned_docs]
    #Trigrams
    if do_trigram:
        trigram = gensim.models.Phrases(output, min_count= min_count_trigram, threshold=threshold_trigram) 
        trigram_mod = gensim.models.phrases.Phraser(trigram)
        output = [trigram_mod[doc] for doc in output]
    return output

In [13]:
# try out on toy collection
toy_normalised = [normalisation(text, lemmatising = True, stemming = True, min_word_len = -1) for text in toy_collection]

In [14]:
# run on true collection
normalised = [normalisation(doc) for doc in collection]

### 3. Extract topics for each document 


In [18]:
def get_top_topics(document, model, min_score=0.8):
    """ 
    Extracting top n topics for each document. 
    Selects the n most likely topics whose p(topic|document) sum to min_score.
    """
    # inserting the document in the model
    new_doc = model.make_doc(document)
    _,_ = model.infer(new_doc)
    # ordering from most probable topic to least one 
    dist = new_doc.get_topic_dist()
    indices = np.flip(np.argsort(dist))
    score = 0
    indices_kept = []
    for index in indices:
        if score > min_score: break
        score += dist[index]
        indices_kept.append(index)
    return indices_kept

In [19]:
# trying on toy collection
get_top_topics(toy_normalised[0], model, 0.5)

[61, 41, 6]

In [22]:
# extracting topics for the original collection
docs2topics = [get_top_topics(doc, model) for doc in normalised]

In [23]:
# enriching original dataframe with the topics list x document 
# Note: the following command will only work if the order of documents is the same for the collection and the docs-topics list
enriched = data
enriched["topics"] = docs2topics

In [25]:
enriched.head(3)

,abstract,id,title,topics
0,The goal of this note is to introduce new clas...,188444.0,Asymptotic versions for operators and operator...,"[105, 15, 107]"
1,We will review a Lemma published by Ran Raz in...,188623.0,Some remarks on a lemma of Ran Raz,"[105, 13, 92]"
2,China’s growing influence in Europe has the po...,346708.0,China as a Stress Test for Europe’s Coherence,"[24, 15, 62, 111]"


In [26]:
# exploding the dataframe (one row for each document-topic pair)
enriched=enriched.explode("topics")
enriched.columns = ["abstract","publication_id","publication_title","topic_id"]

In [27]:
enriched.head(3)

,abstract,publication_id,publication_title,topic_id
0,The goal of this note is to introduce new clas...,188444.0,Asymptotic versions for operators and operator...,105
0,The goal of this note is to introduce new clas...,188444.0,Asymptotic versions for operators and operator...,15
0,The goal of this note is to introduce new clas...,188444.0,Asymptotic versions for operators and operator...,107


In [28]:
# save to .csv 
file_name="abstract+topic.csv"
enriched.to_csv(file_name,index=False)

### 4. Extract words for each topic

In [29]:
def get_top_words(topic, model, min_score=0.8):
    """
    Extracting top n words for each document. 
    Selects the n most likely words whose p(word|topic) sum to min_score.
    """
    dist = model.get_topic_word_dist(topic)
    indices = np.flip(np.argsort(dist))
    score = 0
    word_kept = []
    for index in indices:
        if score > min_score: break
        score += dist[index]
        word_kept.append(model.used_vocabs[index])
    return word_kept

In [33]:
# trying it out
get_top_words(2, model, 0.25)

['tc',
 'spectrum',
 'peak',
 'ce',
 'ab',
 'spectral',
 'band',
 'deriv',
 'sa',
 'transit',
 'experi',
 'signal',
 'inclus',
 'cen',
 'atc',
 'pin',
 'nm',
 'specif_heat']

In [31]:
num_topics = 150

In [34]:
topics2words = [get_top_words(i, model, min_score=0.25) for i in range(num_topics)]

In [36]:
# creating new topic dataframe
topics_df = pd.DataFrame({"TopicID":range(num_topics),"TopicWords":topics2words})

In [37]:
topics_df.head(3)

,TopicID,TopicWords
0,0,"[flow, fluid, transport, pressur, veloc, measu..."
1,1,"[frequenc, nois, signal, oscil, pul, modul, hz..."
2,2,"[tc, spectrum, peak, ce, ab, spectral, band, d..."


In [38]:
# save to .csv 
file_name="topics.csv"
topics_df.to_csv(file_name,index=False)

### Export to Neo4j

Topic nodes: 
- TopicID (long)
- Words (list(str)) 
<br>
<br>


        #Defining the topic nodes
        CREATE CONSTRAINT ON (c:Topic) ASSERT c.ID IS UNIQUE;
        
        #Loading the topic nodes 
        LOAD CSV WITH HEADERS FROM "file:///topics.csv" AS line
        WITH line where line.TopicID IS NOT NULL
        MERGE (t: Topic {ID: line.TopicID})
        SET t.words= line.TopicWords);
        
        #Loading document<->topic relationships
        LOAD CSV WITH HEADERS FROM "file:///abstract+topic.csv" AS line
        MATCH (p:Publication {id:line.publication_id}),
               (t:Topic {code:line.topic_id})
        MERGE (p)-[:IS_ABOUT]->(t)
        MERGE (t)-[:IS_IN]->(p);